# Artificial and Computational Intelligence Assignment 1

# Problem solving by Uninformed & Informed Search

---

**PEAS Description**

---

- **Performance Measure**:
  - **Minimize the total travel distance** while ensuring all blood banks are visited exactly once.
  - Ensure blood is **delivered to the hospital**.
  - **Time Efficiency**, **optimized resource utilization** and **Realiable delivery** are other key performance measure.

- **Environment**:
  - The region includes **multiple blood banks** and **one hospital**.
  - The given environment is **static** and **fully observable**.

- **Actuators**:
  - Drone's **motors and propellers** to control the flight
  - Navigation system of the drone with **gyroscope and accelerometer** to adjust direction and altitude of the drone.
  - **Cargo holder** - a safe mechanism to pick up, carry and release the packages. It could be mechanical arms with motors.
  - **landing Gear**

- **Sensors**:
  - **GPS** or other **location-detection systems** to track the drone's position.
  - **Lidar** or **Camera** for navigating, obstacle avoidance and measure distances.
  - **GSM sensor** in case of cellular communication or other wireless sensors.

---

### **Properties of Task Environment**

---

1. **Observable (Fully/Partially):**
   - **Partially Observable:** The environment is partially observable because the drone might not have complete, real-time information about all variables at all times. For example, weather conditions, traffic (air or ground), or other emergencies may not be fully predictable or observable from the drone's perspective.

2. **Deterministic/Stochastic:**
   - **Stochastic:** The environment is stochastic as it involves uncertainty and randomness. Factors like weather changes, unforeseen obstacles, or variable traffic conditions can influence the outcomes of the drone's actions, making the environment unpredictable.

3. **Episodic/Sequential:**
   - **Sequential:** The environment is sequential because the drone’s actions are dependent on previous decisions. For instance, choosing a particular blood bank first affects the next available options and the overall route efficiency.

4. **Static/Dynamic:**
   - **Dynamic:** The environment is dynamic. It changes over time due to external factors such as weather conditions, air traffic, or even changes in the urgency of the blood delivery. The drone must adapt to these changes in real time.

5. **Discrete/Continuous:**
   - **Continuous:** The environment is continuous because the drone operates in a real-world space where variables like position, speed, and environmental conditions vary continuously rather than in discrete steps.

6. **Single-agent/Multi-agent:**
   - **Single-agent:** The environment is primarily single-agent concerning the Blood Supply Drone. Although the drone may interact with other agents (like air traffic control), the decision-making process for optimizing the route is focused on a single agent (the drone itself).

---

# 1. Initial state with dynamic inputs.

# 2. Matrix for transition and cost

In [1]:
import time
from collections import deque
import tracemalloc

In [2]:
''' This function takes in all the user inputs making it dynamic to different problems.
Input:
Number of nodes Excluding the destination (Hospital) node.
Name for each node (A, B, C ....)
Distance between each nodes in numbers only.

Output:
Distances
Nodes
Hospital (destination node).
'''
def get_dynamic_inputs():
    distances = {}
    nodes = []

    num_nodes = int(input("Enter the total number of blood banks: "))

    # collect all the node's name in the graph.
    for i in range(num_nodes):
        node_name = input(f"Enter the name of blood bank {i+1}: ").strip()
        while not node_name:
            print("Name cannot be empty. Please enter a valid name.")
            node_name = input(f"Enter the name of blood bank {i+1}: ").strip()
        nodes.append(node_name)


    hospital = 'Hospital'

    # Collect the distance between the nodes
    for i in range(len(nodes)):
      node1 = nodes[i]
      for j in range(i+1, len(nodes)):
        node2 = nodes[j]
        distance = input(f"Enter the distance between {node1} and {node2} (press enter if path does not exist): ").strip()
        if distance:
          try:
            distance = float(distance)
            if distance >= 0:
              distances[(node1, node2)] = distance
              distances[(node2, node1)] = distance
            else:
              print("Distance must be non-negative. Skipping this path.")
          except ValueError:
            print("Invalid distance. Skipping this path.")
      distance = input(f"Enter the distance between {node1} and 'Hospital' (press enter if path does not exist): ").strip()
      if distance:
        try:
          distance = float(distance)
          if distance >= 0:
            distances[(node1, 'Hospital')] = distance
            distances[('Hospital',node1)] = distance
          else:
              print("Distance must be non-negative. Skipping this path.")
        except ValueError:
          print("Invalid distance. Skipping this path.")


    print("Distance dictionary after adding user input:", distances)
    print(nodes)
    return distances, nodes, hospital



# 3. Transition model/Successor function

In [3]:
''' This function takes in all the user inputs making it dynamic to different problems.
Input:
Current node - Drone's present node.
Path - current path follwed by drone.
blood_banks - list of all the blood banks given by user.
distances - Graph of all nodes with destination hospital.

Output:
next node
'''
# Successor function to generate next states
def successors(current_node,  path, blood_banks, distances):
    if len(path) == len(blood_banks):  # All blood banks visited, add hospital as the next node
        return ['Hospital']
    return [bank for bank in blood_banks if bank not in path]

# 4. Goal test

1. Find the shortest path from any blood bank to hospital.
2. Find the optimal path minimizing the total travel distance while ensuring that all blood banks are visited exactly once.
Invoke BFS, IDAstar

As per the problem Goal 2 is supposed to be found, goal 1 is additional.

In [4]:
# Function to select the goal (included so as to make the goal dynamic)
def select_goal():
    global goal
    print("Select the goal to be achieved:")
    goal = int(input('Enter 1 if the goal is to find shortest path to hospital\nEnter 2 if the goal is to find path by visiting all the nodes: '))
    while goal not in [1, 2]:
        print("Invalid selection. Please enter 1 or 2.")
        goal = int(input('Enter 1 if the goal is to find shortest path to hospital\nEnter 2 if the goal is to find path by visiting all the nodes: '))
    return goal

# Goal test function
# 1: To check the shortest path from selected blood bank to hospital
# 2: To To find path by visiting all the blood banks


def goal_test(current_node, target_node, path, blood_banks, goal, distances):
    if goal == 1:
        return current_node == target_node
    elif goal == 2:
        return len(path) == len(blood_banks)+1 # plus one to compensate for Hospital as separate node since not included in the blood banks

# Other supporting functions

In [5]:
# Function to calculate average distance to unvisited blood banks
# Heuristic function: Average distance to unvisited blood banks

def average_distance_to_unvisited(current_node, unvisited_banks, distances):
    if not unvisited_banks:
        return 0
    total_distance = 0
    count = 0
    for bank in unvisited_banks:
        if (current_node, bank) in distances:
            total_distance += distances[(current_node, bank)]
            count += 1
        elif (bank, current_node) in distances:
            total_distance += distances[(bank, current_node)]
            count += 1
    return total_distance / count if count > 0 else 0

def calculate_total_distance(path, distances):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distances[(path[i], path[i + 1])]
    return total_distance

'''
    Find the optimal path for the Blood Supply Vehicle.

    Inputs:
    - blood_banks -  List of blood bank.
    - distances - Graph (nested dictionary) of distances between nodes.
    - goal: 1 or 2 as described in "select goal" function.
    - target_node: hospital.
    - bfsRida: Search strategy ('BFS' or 'IDA*').

    Outputs:
    - optimal_path: List of nodes in the optimal path.
    - min_distance: Total distance of the optimal path.

'''

def find_optimal_path(blood_banks, distances, goal, target_node, bfsRida):
    min_distance = float('inf')
    optimal_path = []

    for start_node in blood_banks:
        if bfsRida == 'BFS':
          path,_  = BFS(start_node, blood_banks,distances, goal, target_node)
        elif bfsRida == 'IDA*':
          path = ida_star(start_node, target_node, distances, blood_banks, goal)
        if path:
            total_distance = calculate_total_distance(path, distances)
            print(path, total_distance)
            if total_distance < min_distance:
                min_distance = total_distance
                optimal_path = path

    return optimal_path, min_distance


# 5. BFS - Bredth First Search Function.

In [6]:
'''
Implementation of BFS algorithm
input:
- Start node: intial node
- blood_banks - list of all blood banks
- distances: Graph (nested dictionary) of distances between the nodes
- target node: Hospital

output:
Optimal path and distance travelled as per BFS algorithm
'''
def BFS(start, blood_banks, distances, goal, target_node):
    queue = deque([(start, [start], 0)])  # (current_node, path, total_distance)
    min_distance = float('inf')
    optimal_path = []

    while queue:
        current_node, path, total_distance = queue.popleft()
        if goal_test(current_node, target_node, path, blood_banks, goal, distances):
            if total_distance < min_distance:
                min_distance = total_distance
                optimal_path = path
            continue

        for neighbor in successors(current_node, path, blood_banks, distances):
            distance = distances.get((current_node, neighbor), distances.get((neighbor, current_node), float('inf')))
            queue.append((neighbor, path + [neighbor], total_distance + distance))

    return optimal_path, min_distance

# 6. IDAStar - Iterative Depth A star function

In [7]:
'''
Implementation of IDA* algorithm
input:
- Start node: intial node
- blood_banks - list of all blood banks
- distances: Graph (nested dictionary) of distances between the nodes
- target node: Hospital

output:
Optimal path and distance travelled as per IDA* algorithm
'''

# Iterative Deepening A* (IDA*) search for shortest path
def ida_star(start, target_node, distances, nodes, goal):
    def search(path, g, bound, nodes, goal):
        current_node = path[-1]
        f = g + average_distance_to_unvisited(current_node, target_node, distances)
        if f > bound:
            return f, None
        if goal_test(current_node, target_node, path, nodes, goal, distances):
            return g, path
        min_bound = float('inf')
        for neighbor in successors(current_node, path, nodes, distances):
            next_g = g + distances.get((current_node, neighbor), distances.get((neighbor, current_node), float('inf')))
            path.append(neighbor)
            t, result = search(path, next_g, bound, nodes, goal)
            if result:
                return t, result
            if t < min_bound:
                min_bound = t
            path.pop()
        return min_bound, None

    bound = average_distance_to_unvisited(start, set(nodes) - {start}, distances)
    path = [start]
    while True:
        t, result = search(path, 0, bound, nodes, goal)
        if result:
            return result
        if t == float('inf'):
            return None
        bound = t


# 7. Function & call to get inputs (start/end state) Display the possible states to choose from.

In [10]:
'''    distances = {
        ('A', 'B'): 5,
        ('A', 'C'): 8,
        ('B', 'C'): 7,
        ('B', 'D'): 6,
        ('B', 'E'): 10,
        ('B', 'Hospital'): 8,
        ('C', 'F'): 12,
        ('E', 'F'): 9,
        ('D', 'Hospital'): 10,
        ('E', 'Hospital'): 18,
        ('B', 'A'): 5,
        ('C', 'A'): 8,
        ('C', 'B'): 7,
        ('D', 'B'): 6,
        ('E', 'B'): 10,
        ('Hospital', 'B'): 8,
        ('F', 'C'): 12,
        ('F', 'E'): 9,
        ('Hospital', 'D'): 10,
        ('Hospital', 'E'): 18
    }'''
distances, nodes, target_node = get_dynamic_inputs()

Enter the total number of blood banks: 6
Enter the name of blood bank 1: A
Enter the name of blood bank 2: B
Enter the name of blood bank 3: C
Enter the name of blood bank 4: D
Enter the name of blood bank 5: E
Enter the name of blood bank 6: F
Enter the distance between A and B (press enter if path does not exist): 5
Enter the distance between A and C (press enter if path does not exist): 8
Enter the distance between A and D (press enter if path does not exist): 
Enter the distance between A and E (press enter if path does not exist): 
Enter the distance between A and F (press enter if path does not exist): 
Enter the distance between A and 'Hospital' (press enter if path does not exist): 
Enter the distance between B and C (press enter if path does not exist): 7
Enter the distance between B and D (press enter if path does not exist): 6
Enter the distance between B and E (press enter if path does not exist): 10
Enter the distance between B and F (press enter if path does not exist): 


# 8. Slect the goal
1. Find the shortest path from any blood bank to hospital.
2. Find the optimal path minimizing the total travel distance while ensuring that all blood banks are visited exactly once.
Invoke BFS, IDAstar

As per the problem Goal 2 is supposed to be found, goal 1 is additional and can be ignored.

In [11]:
goal = select_goal()
if goal == 1:
  start_node = input("Enter the starting node: ")
  nodes.append(target_node)
  tracemalloc.start()
  start_time_ida = time.time()
  optimal_path_ida = ida_star(start_node, target_node, distances, nodes, goal)
  end_time_ida = time.time()
  min_distance_ida = calculate_total_distance(optimal_path_ida, distances) if optimal_path_ida else float('inf')
  memory_ida, _ = tracemalloc.get_traced_memory()
  tracemalloc.stop()

  tracemalloc.start()
  start_time_bfs = time.time()
  optimal_path_bfs, min_distance_bfs  = BFS(start_node, nodes,distances, goal, target_node)
  end_time_bfs = time.time()
  memory_bfs, _ = tracemalloc.get_traced_memory()
  tracemalloc.stop()

elif goal == 2:
  tracemalloc.start()
  start_time_ida = time.time()
  optimal_path_ida, min_distance_ida = find_optimal_path(nodes,distances, goal, target_node, 'IDA*')
  end_time_ida = time.time()
  memory_ida, _ = tracemalloc.get_traced_memory()
  tracemalloc.stop()

  tracemalloc.start()
  start_time_bfs = time.time()
  optimal_path_bfs, min_distance_bfs = find_optimal_path(nodes,distances, goal, target_node, 'BFS')
  end_time_bfs = time.time()
  memory_bfs, _ = tracemalloc.get_traced_memory()
  tracemalloc.stop()

# Print results and comparison
print("IDA* results:")
print(f"Optimal path: {' -> '.join(optimal_path_ida)}" if optimal_path_ida else "No path found.")
print(f"Total distance: {min_distance_ida}")
print(f"Time taken: {end_time_ida - start_time_ida:.6f} seconds")
print(f"Memory usage: {memory_ida / 1024:.2f} KB")

print("\nBFS results:")
print(f"Optimal path: {' -> '.join(optimal_path_bfs)}" if optimal_path_bfs else "No path found.")
print(f"Total distance: {min_distance_bfs}")
print(f"Time taken: {end_time_bfs - start_time_bfs:.6f} seconds")
print(f"Memory usage: {memory_bfs / 1024:.2f} KB")

Select the goal to be achieved:
Enter 1 if the goal is to find shortest path to hospital
Enter 2 if the goal is to find path by visiting all the nodes: 2
['A', 'C', 'F', 'E', 'B', 'D', 'Hospital'] 55.0
['D', 'B', 'A', 'C', 'F', 'E', 'Hospital'] 58.0
['E', 'F', 'C', 'A', 'B', 'D', 'Hospital'] 50.0
['A', 'C', 'F', 'E', 'B', 'D', 'Hospital'] 55.0
['D', 'B', 'A', 'C', 'F', 'E', 'Hospital'] 58.0
['E', 'F', 'C', 'A', 'B', 'D', 'Hospital'] 50.0
IDA* results:
Optimal path: E -> F -> C -> A -> B -> D -> Hospital
Total distance: 50.0
Time taken: 0.018653 seconds
Memory usage: 10.46 KB

BFS results:
Optimal path: E -> F -> C -> A -> B -> D -> Hospital
Total distance: 50.0
Time taken: 0.018134 seconds
Memory usage: 15.98 KB


# 9. Comparitive analysis

In [12]:
# Determine the best algorithm
best_algorithm = "IDA*" if min_distance_ida < min_distance_bfs else "BFS" if min_distance_bfs < min_distance_ida else "Both are equally optimal"
print(f"\nBest algorithm based on distance: {best_algorithm}")

if end_time_ida - start_time_ida < end_time_bfs - start_time_bfs:
    print("IDA* is faster.")
else:
    print("BFS is faster.")

if memory_ida < memory_bfs:
    print("IDA* uses less memory.")
else:
    print("BFS uses less memory.")


Best algorithm based on distance: Both are equally optimal
BFS is faster.
IDA* uses less memory.


# Program end here.